In [40]:
import pandas as pd
from pandas.io import sql
import sqlite3
import numpy as np

In [12]:
conn=sqlite3.connect("../datasets/enron.db")
c=conn.cursor()

### This is how we get table names of database

In [17]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print c.fetchall()

[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]


In [14]:
results = c.execute('SELECT * FROM EmployeeBase LIMIT 5')
results.fetchall()

[(1,
  u'John Arnold',
  u'Forestry',
  u'ENA Gas Financial',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (2,
  u'Harry Arora',
  u'Forestry',
  u'ENA East Power',
  u'VP Trading',
  u'Male',
  u'Senior'),
 (3,
  u'Robert Badeer',
  u'Forestry',
  u'ENA West Power',
  u'Mgr Trading',
  u'Male',
  u'Junior'),
 (4,
  u'Susan Bailey',
  u'Legal',
  u'ENA Legal',
  u'Specialist Legal',
  u'Female',
  u'Junior'),
 (5,
  u'Eric Bass',
  u'Forestry',
  u'ENA Gas Texas',
  u'Trader',
  u'Male',
  u'Junior')]

In [18]:
results = c.execute('SELECT * FROM MessageBase LIMIT 5')
results.fetchall()

[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138),
 (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138),
 (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138),
 (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138),
 (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)]

In [19]:
results = c.execute('SELECT * FROM RecipientBase LIMIT 5')
results.fetchall()

[(1, 1, 59), (2, 1, 15), (3, 1, 15), (4, 1, 109), (4, 2, 49)]

In [23]:
employees = pd.read_sql('SELECT * FROM EmployeeBase', con=conn)
employees.head(5)

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [24]:
messages = pd.read_sql('SELECT * FROM MessageBase', con=conn)
messages.head(5)

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [25]:
recipients = pd.read_sql('SELECT * FROM RecipientBase', con=conn)
recipients.head(5)

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


In [26]:
employees.count()

eid               156
name              156
department        156
longdepartment    156
title             156
gender            156
seniority         156
dtype: int64

In [28]:
messages['dateime'] = pd.to_datetime(messages.unix_time, unit='s')

In [30]:
messages.head()

,mid,filename,unix_time,subject,from_eid,dateime
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138,1998-11-13 04:07:00
1,2,taylor-m/sent/17,911459940,Indemnification,138,1998-11-19 07:19:00
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138,1998-11-19 08:24:00
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138,1998-11-24 02:23:00
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138,1998-11-30 03:22:00


In [31]:
messages.dateime.describe()

count                   21635
unique                  21309
top       1999-12-27 03:21:00
freq                        4
first     1998-11-13 04:07:00
last      2002-06-21 13:37:34
Name: dateime, dtype: object

In [32]:
print messages.dateime.min(), messages.dateime.max()

1998-11-13 04:07:00 2002-06-21 13:37:34


In [49]:
piv = recipients.pivot_table(index='mid', values='rno', aggfunc=np.max)
piv.value_counts()

1     14985
2      2962
3      1435
4       873
5       711
6       180
7       176
8        61
13       57
11       47
12       33
10       29
15       28
9        24
14       11
16        9
21        2
17        2
57        2
22        1
52        1
20        1
55        1
19        1
24        1
18        1
49        1
Name: rno, dtype: int64

In [50]:
employees.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


Are there more Men or Women employees?

In [54]:
employees.pivot_table(index='gender', values='eid', aggfunc=np.count_nonzero).reset_index()

,gender,eid
0,Female,43
1,Male,113


How is gender distributed across departments?

In [53]:
employees.pivot_table(index=['department', 'gender'], values='eid', aggfunc=np.count_nonzero).reset_index()

,department,gender,eid
0,Forestry,Female,10
1,Forestry,Male,50
2,Legal,Female,13
3,Legal,Male,12
4,Other,Female,20
5,Other,Male,51


Who is sending more emails? Men or Women?

In [65]:
messages.head(2)

,mid,filename,unix_time,subject,from_eid,dateime
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138,1998-11-13 04:07:00
1,2,taylor-m/sent/17,911459940,Indemnification,138,1998-11-19 07:19:00


In [72]:
senders = messages.pivot_table(index='from_eid', values='mid', aggfunc=np.count_nonzero).reset_index()

In [75]:
sendersinfo = senders.join(employees, on='from_eid', how='left')

In [77]:
sendersinfo.pivot_table(index='gender', values='mid', aggfunc=np.sum).reset_index()

,gender,mid
0,Female,5685
1,Male,15754


What's the average number of emails sent by each gender?

In [78]:
sendersinfo.pivot_table(index='gender', values='mid', aggfunc=np.mean).reset_index()

,gender,mid
0,Female,138.658537
1,Male,143.218182


Are there more Juniors or Seniors?

In [79]:
employees.pivot_table(index='seniority', values='eid', aggfunc=np.count_nonzero).reset_index()

,seniority,eid
0,Junior,82
1,Senior,74


Who is sending more emails? Juniors or Seniors?

In [80]:
sendersinfo.pivot_table(index='seniority', values='mid', aggfunc=np.sum).reset_index()

,seniority,mid
0,Junior,11351
1,Senior,10088


Which department is sending more emails? How does that relate with the number of employees in the department?

In [81]:
sendersinfo.pivot_table(index='department', values='mid', aggfunc=np.sum).reset_index()

,department,mid
0,Forestry,7130
1,Legal,3521
2,Other,10788


Who are the top 3 senders of emails? (people who sent out the most emails)

In [86]:
sendersinfo.pivot_table(index='eid', values='mid', aggfunc=np.sum).reset_index().sort_values(by=['mid'], ascending=[False]).head(3)

,eid,mid
19,21.0,1597
55,60.0,1379
116,121.0,1142


Who is receiving more emails? Men or Women?

In [88]:
recipients.head(2)

,mid,rno,to_eid
0,1,1,59
1,2,1,15


In [110]:
top_receivers = recipients.to_eid.value_counts().reset_index()
top_receivers = top_receivers.join(employees, on='index', how='left')

In [111]:
top_receivers.pivot_table(index='gender', values='to_eid', aggfunc=np.sum).reset_index()

,gender,to_eid
0,Female,8496
1,Male,29554


Who is receiving more emails? Juniors or Seniors?

In [107]:
top_receivers.pivot_table(index='seniority', values='to_eid', aggfunc=np.sum).reset_index()

,seniority,to_eid
0,Junior,19843
1,Senior,18207


Which department is receiving more emails? How does that relate with the number of employees in the department?

In [108]:
top_receivers.pivot_table(index='department', values='to_eid', aggfunc=[np.sum, np.count_nonzero]).reset_index()

,department,sum,count_nonzero
0,Forestry,16327,58
1,Legal,5372,25
2,Other,16351,70


Who are the top 5 receivers of emails? (people who received the most emails)

In [112]:
top_receivers.head(5)

,index,to_eid,eid,name,department,longdepartment,title,gender,seniority
0,131,1797,132.0,Joe Stepenovitch,Other,ENA East Power,Specialist,Male,Junior
1,122,1730,123.0,Hunter S. Shively,Forestry,ENA Gas Central,VP Trading,Male,Senior
2,138,1477,139.0,Michael E. Taylor,Forestry,Energy Operations,Trader,Male,Junior
3,61,1290,62.0,Peter F. Keavey,Forestry,ENA Gas East,Mgr Trading,Male,Junior
4,120,1173,121.0,Jeffrey A. Shankman,Other,EWS,COO,Male,Senior


In [113]:
recipients.head(2)

,mid,rno,to_eid
0,1,1,59
1,2,1,15


In [121]:
messages.head(2)

,mid,filename,unix_time,subject,from_eid,dateime
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138,1998-11-13 04:07:00
1,2,taylor-m/sent/17,911459940,Indemnification,138,1998-11-19 07:19:00


In [116]:
mass_emails = recipients.pivot_table(index='mid', values='to_eid', aggfunc=np.sum).reset_index()

In [118]:
mass_emails.head(5)

,mid,to_eid
0,1,59
1,2,15
2,3,15
3,4,337
4,5,98


In [122]:
mass_emails.join?

In [128]:
a = mass_emails.merge(messages, on='mid', how='left')
a.join(employees, on='from_eid', how='left')['eid'].value_counts()

21.0     1597
60.0     1379
121.0    1142
132.0     859
139.0     658
131.0     654
40.0      443
146.0     409
92.0      403
96.0      395
35.0      366
66.0      361
68.0      354
62.0      354
2.0       341
118.0     322
49.0      322
47.0      318
6.0       318
44.0      316
152.0     302
123.0     295
25.0      290
93.0      273
75.0      260
70.0      249
138.0     244
145.0     237
150.0     227
119.0     224
         ... 
56.0       12
36.0       10
142.0      10
15.0       10
125.0      10
137.0      10
23.0       10
129.0      10
13.0        9
14.0        9
97.0        9
65.0        9
111.0       8
133.0       8
51.0        8
29.0        7
72.0        7
17.0        7
104.0       7
98.0        6
84.0        6
90.0        6
151.0       5
106.0       5
24.0        5
71.0        4
100.0       3
9.0         2
87.0        1
22.0        1
Name: eid, dtype: int64